# Decision Trees

In [ ]:
#importing libraries
import numpy as np
import pandas as pd

In [ ]:
eps = np.finfo(float).eps
eps

Here `eps` is the smallest representable number. At times we get $log(0)$ or $0$. We will use `eps` to avoid underflow

## Data

In [ ]:
data = {'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain' ],
        'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
        'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
        'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak',  'Strong', 'Strong', 'Weak', 'Strong'],
        'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No',  'Yes', 'Yes',  'Yes', 'Yes', 'Yes', 'No' ]}
df = pd.DataFrame(data)

In [ ]:
df.shape

In [ ]:
df.values

In [ ]:
df.keys()

In [ ]:
target = df.keys()[-1]
target

In [ ]:
df[target].unique()

In [ ]:
df[target].value_counts()

## Implementation

### Calculating entropy of the whole dataset

In [ ]:
def find_entropy_whole(df):
    #last column in the dataframe is the target variable
    target = df.keys()[-1]

    #initialisation
    overall_entropy = 0

    #possible values of target
    values_in_target = df[target].unique()

    for value in values_in_target:
        p = df[target].value_counts()[value] / len(df[target])
        overall_entropy += -p*np.log2(p)
    
    return overall_entropy

find_entropy_whole(df)

### Calculating entropy of an attribute

In [ ]:
def find_entropy_of_attribute(df, attribute):
    target = df.keys()[-1]
    values_in_target = df[target].unique()
    values_in_attribute = df[attribute].unique()

    entropy_attribute = 0

    for attribute_value in values_in_attribute:
        overall_entropy = 0

        for target_value in values_in_target:
            num = len(df[attribute][df[attribute] == attribute_value][df[target] == target_value])
            den = len(df[attribute][df[attribute] == attribute_value])

            p = num/(den + eps)
            overall_entropy += -p * np.log2(p + eps) 
        p2 = den / len(df)
        entropy_attribute += -p2 * overall_entropy

    return abs(entropy_attribute) 

In [ ]:
for i in df.keys()[:-1]:
    print(f'Entropy of attribute \'{i}\' is :', find_entropy_of_attribute(df, i))

### Finding the best attribute

In [ ]:
def find_best_attribute_to_divide(df):
    #Information gain
    IG = []

    all_attribute_names = df.keys()[:-1]

    for attr in all_attribute_names:
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attr))
    
    index_of_attr_with_max_IG = np.argmax(IG)

    best_attribute = all_attribute_names[index_of_attr_with_max_IG]

    return best_attribute

In [ ]:
find_best_attribute_to_divide(df)

### Building decision tree

In [ ]:
def buildTree(df, tree=None):

    target = df.keys()[-1]

    #Get attribute with mximum information gain
    node = find_best_attribute_to_divide(df)

    # get distinct value of that attribute
    attValue = np.unique(df[node])

    # Create an empty dictionary to create tree
    if tree is None:
        tree = {}
        tree[node] = {}

    # We make loop to construct a tree by calling this function recursively
    for value in attValue:

        subtable = df[df[node] == value].reset_index(drop=True)
        clValue, counts = np.unique(subtable[target], return_counts=True)

        if len(counts) == 1: #Checking purity of subset
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) #recursive function call

    return tree

buildTree(df)

ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it ever back tracks to reconsider this choice. Therefore it is susceptible to the usual risks of hill-climbing search without backtracking: converging to a locally optimal solutions that are not globally optimal.